# As the exploratory part was already done in the original file, I've cleared it on this version, to jump strainght to the point

In [18]:
#standard imports
import numpy as np 
import pandas as pd 
import seaborn as sns
import time
import copy
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#loading datasets
features=pd.read_csv("C:\\Users\\Paulo\\Desktop\\walmart-git\\wallmart-sales-forecasting\\features.csv")
samples=pd.read_csv("C:\\Users\\Paulo\\Desktop\\walmart-git\\wallmart-sales-forecasting\\sampleSubmission.csv")
stores=pd.read_csv("C:\\Users\\Paulo\\Desktop\\walmart-git\\wallmart-sales-forecasting\\stores.csv")
test=pd.read_csv("C:\\Users\\Paulo\\Desktop\\walmart-git\\wallmart-sales-forecasting\\test.csv")
train=pd.read_csv("C:\\Users\\Paulo\\Desktop\\walmart-git\\wallmart-sales-forecasting\\train.csv") 

In [19]:
### I have condensed all modifications on this single cell. For more information or explanation please refere to: 
# https://github.com/p-caixeta/wallmart-sales-forecasting/blob/master/walmart-recruiting-store-sales-forecasting.ipynb
    

#getting all information on one singel DF

#merging
storesComFeatures=pd.merge(stores,features, on="Store",how="inner")

#adding sales
treino_dados_totais=pd.merge(storesComFeatures,train,how="inner",on=["Store","Date","IsHoliday"]).reset_index(drop=True)
treino_dados_totais.sort_values(by=["Store","Dept","Date"])
teste_dados_totais=pd.merge(storesComFeatures,test,how="inner",on=["Store","Date","IsHoliday"]).reset_index(drop=True)
teste_dados_totais.sort_values(by=["Store","Dept","Date"])

#correncting Date and IsHolliday formats
treino_dados_totais["Date"]=     pd.to_datetime(treino_dados_totais["Date"])
treino_dados_totais["IsHoliday"]=pd.get_dummies(treino_dados_totais["IsHoliday"],drop_first=True)
teste_dados_totais["Date"]=      pd.to_datetime(teste_dados_totais["Date"])
teste_dados_totais["IsHoliday"]= pd.get_dummies(teste_dados_totais["IsHoliday"],drop_first=True)

#creating a "week" and a "year" column, to follow sales throughout the year
treino_dados_totais["year"]=   treino_dados_totais["Date"].dt.year
treino_dados_totais["week"]=treino_dados_totais["Date"].dt.week
teste_dados_totais["year"]=    teste_dados_totais["Date"].dt.year
teste_dados_totais["week"]= teste_dados_totais["Date"].dt.week

#getting numeric dummies for store type
treinoDummies=pd.get_dummies (treino_dados_totais["Type"],prefix="Store_Type")
testeDummies= pd.get_dummies (teste_dados_totais["Type"],prefix="Store_Type")

treino_dados_totais=pd.concat([treino_dados_totais, treinoDummies], axis=1)
teste_dados_totais= pd.concat([teste_dados_totais, testeDummies], axis=1)

treino_dados_totais.drop("Type",axis=1,inplace=True)
teste_dados_totais.drop ("Type",axis=1,inplace=True)

#as Markdowns are discounts, makes sense to fill nulls with 0, since 0 will indicate no discounts
treino_dados_totais["MarkDown4"].fillna(0,inplace=True)
treino_dados_totais["MarkDown5"].fillna(0,inplace=True)
teste_dados_totais["MarkDown4"].fillna(0,inplace=True)
teste_dados_totais["MarkDown5"].fillna(0,inplace=True)

del storesComFeatures,features,stores,test,train

teste_dados_totais.drop(["Fuel_Price","MarkDown1","MarkDown2","MarkDown3","Temperature","CPI"],axis=1,inplace=True)
treino_dados_totais.drop(["Fuel_Price","MarkDown1","MarkDown2","MarkDown3","Temperature","CPI"],axis=1,inplace=True)

teste_dados_totais["Unemployment"].fillna(method="ffill",inplace=True) 

#removing date
teste_final=teste_dados_totais.drop("Date",axis=1)
teste_final.sort_values(by=["Store","Dept"],inplace=True)  #to ccorrect the size
treino_final=treino_dados_totais.drop("Date",axis=1)

#creating WMAE function
def WMAE(df, real, pred):
    peso = df["IsHoliday"].apply(lambda x: 5 if x else 1)
    return np.round(np.sum(peso*abs(real-pred))/(np.sum(peso)), 2)

xtrain,xtest,ytrain,ytest= train_test_split(
    treino_final.drop("Weekly_Sales",axis=1) , treino_final["Weekly_Sales"] , test_size=0.3, random_state=42)

In [22]:
#loop for Random Forest:
print ("Method = RF")
min_erro_rf=1665
#rf=RandomForestRegressor(n_estimators=n,
#                        max_depth=depth,
#                        min_samples_split=split,
#                        min_samples_leaf=leaf,
#                        random_state=42)
#rf.fit(xtrain,ytrain)
#pred=rf.predict(xtest)
#erro=WMAE(xtest,ytest,pred)
#print ("Best result=",min_erro_rf)
#print ("Best parameters=",melhor_param_rf)

#best "brute force" result: WMAE 1665, parameter = 500,100,1,2

Method = RF


In [23]:
#making the submission
#best method: Random Forest
#melhor_param_rf = [500, 100, 1, 2]
#rf=RandomForestRegressor(n_estimators=melhor_param_rf[0],
#                        max_depth=melhor_param_rf[1],
#                        min_samples_leaf=melhor_param_rf[2],
#                        min_samples_split=melhor_param_rf[3],
#                        random_state=42)
#rf.fit(xtrain,ytrain)
#previsto=rf.predict(teste_final)
#samples["Weekly_Sales"]=previsto
#samples.to_csv('submission.csv',index=False)

# > > **PSO**

Now, I'll implement Particle Swarm Optimization (http://ai.unibo.it/sites/ai.unibo.it/files/u11/pso.pdf) to find the best parameters for Random Forest

In [ ]:
w = 0.729844 # Inertia weight to prevent velocities becoming too large 
c1 = 2.609 # Scaling co-efficient on the social component
c2 = 2.496180 # Scaling co-efficient on the cognitive component
iterations = 200
swarmSize = 200
rd_state=42
dimension=4 # 1= n_estimators, 2= max_depth, 3=min_sample_leaf, 4=min_sample_split

tempo_inicial=time.time()
inicio=time.clock()
## constrains: ##
#     1<= n_estimators <= 1000 
#    10<= max_depth <= 200
#     1<= min_sample_leaf <=20
#     1<= min_sample_split <=20

y=[0]*dimension
x=[0]*swarmSize
v=[0]*swarmSize
for i in range (swarmSize):
    x[i]=copy.deepcopy(y)
    v[i]=copy.deepcopy(y)
melhora=[]
########## INITIAL POPULATION ################  
## n_estimators ##
for i in range(swarmSize):
    x[i][0]=np.random.randint(1,1000)
    v[i][0]=0.01*x[i][0]
## max_depth ##
    x[i][1]=np.random.randint(10,200)
    v[i][1]=0.01*x[i][1]
## min_sample_leaf ##
    x[i][2]=np.random.randint(1,20)
    v[i][2]=0.01*x[i][2]
## min_sample_split ##
    x[i][3]=np.random.randint(1,20)
    v[i][3]=0.01*x[i][3]

pbest=copy.deepcopy(x) 
fpbest=[None]*swarmSize
a=[None]*swarmSize
b=time.clock()
for i in range(swarmSize): 
    rf=RandomForestRegressor(n_estimators=x[i][0],
                        max_depth=x[i][1],
                        min_samples_leaf=x[i][2],
                        min_samples_split=x[i][3],
                        random_state=42)
    rf.fit(xtrain,ytrain)
    A=rf.predict(teste_final)
    fpbest[i]=copy.deepcopy([A])
    a[i]=copy.deepcopy([A])
    print("fim do fit inicial",i,"particula",time.clock())
a.sort()
gbest=copy.deepcopy(a[0])

xbest=copy.deepcopy(x[fpbest.index(gbest)])

gbest_inicial=copy.deepcopy(gbest[0])
print ("\n-----------FIM DA INICIALIZACAO-------------------",
           "\ngbest inicial=",gbest[0],
           "\ntempo gasto:",time.time()-tempo_inicial,"s"
           ,"\ntempo medio por fit:",(time.clock()- b)/swarmSize,
           "s","\n----------------------------------------------------"
           ,"xbest:\n",xbest
           )
########## MAIN LOOP #################
for inte in range(iterations):
    ########## updating generating #################
    b=time.time()
    for j in range(swarmSize):
        r1=np.random.random()            
        r2=np.random.random()
        social = c1 * r1 * (xbest - x[j])
        cognitive = c2 * r2 * (pbest[j] - x[j])
        v[j] = (w * v[j]) + social + cognitive
                #print ("aqui",j,i)
            #print ("aqui2",j,i)
            #print("fim da evoucao da",inte," geracao. Durou:",time.clock()/60,"min")
        ########## comparaÃ§Ã£o do gbest ####################
    for k in range(swarmSize): 
        c=time.clock()
        rf=RandomForestRegressor(n_estimators=x[i][0],
                        max_depth=x[i][1],
                        min_samples_leaf=x[i][2],
                        min_samples_split=x[i][3],
                        random_state=42)
        rf.fit(xtrain,ytrain)
        A=rf.predict(teste_final)
        if A<gbest:
            melhora+=[inte]
            gbest=copy.deepcopy(A)
            xbest=copy.deepcopy(x[k])
        if A<fpbest[k]:
            fpbest[k]=copy.deepcopy(A)
            pbest[k]=copy.deepcopy(x[k])
        else:
            pass
    d=time.clock()
    if inte%10==0 and inte>1:
        print("\nfim da geracao",inte,"com media de","%.5f" % ((d-c)/swarmSize),
                  "s por fit")

        print ("tempo total:",(time.time()-tempo_inicial)/60,"min",#)
                   "\ntempo gasto nesta iteracao:",time.time()-b,"s")

        print ("gbest ate a geracao",inte,"=",gbest,
                   "\n xbest ate geracao:\n",xbest,
                   "\n-----------------------"
                   )

fim da pop inicial
